In [5]:
import os
from pathlib import Path
import bifacial_radiance as br
import numpy as np
import datetime
import pickle
import pandas as pd
import numpy as np
import bifacialvf

# Making folders for saving the simulations
testfolder = os.path.join(os.getcwd(), 'TEMP')

In [35]:
print(br.__version__)
print(bifacialvf.__version__)

0.4.2+143.gd6cf83b.dirty
0.1.8.1


In [9]:
radObj = br.RadianceObj('Setup',testfolder)

lat = 39.7407
lon = -105.1686
sazm = 180 
y = 2 # Collector Width for 1-up

epwfile = radObj.getEPW(lat, lon) 
metData = radObj.readWeatherFile(epwfile)

path = C:\Users\sayala\Documents\GitHub\Studies\AgriPVWorld\TEMP
Getting weather file: USA_CO_Golden-NREL.724666_TMY3.epw
 ... OK!
8760 line in WeatherFile. Assuming this is a standard hourly WeatherFile for the year for purposes of saving Gencumulativesky temporary weather files in EPW folder.
Coercing year to 2021
Saving file EPWs\metdata_temp.csv, # points: 8760
Calculating Sun position for Metdata that is right-labeled  with a delta of -30 mins. i.e. 12 is 11:30 sunpos


In [34]:
# Distance between rows for no shading on Dec 21 at 9 am (Northern Hemisphere) or Jun 21st (Southern Hemisphere)
DD = bifacialvf.vf.rowSpacing(np.round(metData.latitude), sazm, metData.latitude, metData.longitude, metData.timezone, 9, 0.0)
normalized_pitch = DD + np.cos(np.round(metData.latitude) / 180.0 * np.pi)
pitch = normalized_pitch*y
pitch

5.116777543792527